In [32]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [33]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [34]:
df = DataFrame(XLSX.readtable("../../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1228,1.3903,0.1011,0.3652,-2.6994,-1.1768,-0.1044,0.2218,4.9165,0.8232,0.1442
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1239,1.3604,0.0656,0.3354,-2.5776,-1.0987,-0.1294,0.2287,4.1943,0.5297,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0       0.188888
 0.188888  1.0

In [35]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

ret_mean = mean(values(data), dims=1)[1,:]
ret_cov = cov(values(data), dims=1)

T = 25 * freq
n_samples = 10_000
n_assets = length(assets_names)

scenarios = zeros(n_assets, T, n_samples)

for t in 1:T 
    scenarios[:,t, :] = rand(MvNormal(ret_mean, ret_cov), n_samples)
end

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  



,Total_Stock_CPI,Price_10Y_CPI
1,0.0687,0.0182
5,0.0669,0.0176
10,0.0661,0.0176
25,0.0661,0.0177


,Total_Stock_CPI,Price_10Y_CPI
1,0.1263,0.0787
5,0.0554,0.0349
10,0.039,0.0248
25,0.0248,0.0156


,Total_Stock_CPI,Price_10Y_CPI
1,-0.0004,0.0143
5,-0.0018,0.0231
10,-0.0192,-0.0063
25,-0.0277,-0.0176


,Total_Stock_CPI,Price_10Y_CPI
1,0.0388,-0.0129
5,-0.0115,0.014
10,0.03,-0.1037
25,-0.0428,-0.1081


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.2255,-0.1814,-0.1395,-0.016,0.0681,0.1549,0.2731,0.3142,0.3625
5,-0.0602,-0.0405,-0.024,0.0294,0.0668,0.1044,0.1583,0.1756,0.1957
10,-0.0243,-0.0101,0.0022,0.0395,0.0666,0.0923,0.1304,0.1416,0.1567
25,0.0079,0.0169,0.0257,0.0495,0.0661,0.083,0.1069,0.114,0.1219


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1637,-0.1362,-0.1115,-0.0341,0.0176,0.0715,0.1482,0.1728,0.2047
5,-0.0652,-0.0493,-0.0387,-0.0063,0.0173,0.0409,0.075,0.0876,0.0983
10,-0.0398,-0.0307,-0.0232,0.0009,0.0175,0.0346,0.0588,0.0657,0.074
25,-0.0187,-0.0131,-0.0082,0.0071,0.0177,0.0285,0.0433,0.0482,0.0535


In [36]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.173717  0.232394  0.280154  0.339215  0.479585

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 26.0  44.0  61.0  88.0  171.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.14248  0.204298  0.25437  0.319766  0.473124

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 40.0  76.0  111.0  166.0  285.0